In [1]:
import numpy as np
import pickle as pkl
import pandas as pd
from sklearn.cluster import MiniBatchKMeans
from itertools import cycle

import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

plt.rcParams['figure.figsize'] = (8, 8)


In [2]:
df = pd.read_csv('/Users/dilys/Desktop/Harvard_1718_Fall/CS282/cs282-f17-xuefeng-yi-david/data/Sepsis_imp_train.csv')

In [3]:
observations =   ['gender','age','elixhauser','re_admission','SOFA','SIRS','Weight_kg','GCS','HR',
                  'SysBP','MeanBP','DiaBP','Shock_Index','RR','SpO2','Temp_C','FiO2_1','Potassium',
                  'Sodium','Chloride','Glucose','BUN','Creatinine','Magnesium','Calcium','Ionised_Ca',
                  'CO2_mEqL','SGOT','SGPT','Total_bili','Albumin','Hb','WBC_count','Platelets_count',
                  'PTT','PT','INR','Arterial_pH','paO2','paCO2','Arterial_BE','Arterial_lactate','HCO3',
                  'PaO2_FiO2','output_total','output_4hourly','sedation','mechvent','rrt']
# Per homework, 'sedation','mechvent','rrt' are considered observations, not interventions.
interventions = ['input_4hourly_tev','max_dose_vaso']

results = ['died_in_hosp','mortality_90d']

X = df[observations]


In [ ]:
n_clusters = 750
mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, batch_size=100,
                      n_init=10, max_no_improvement=10, verbose=0, init_size=3*n_clusters,
                      random_state=0)
mbk.fit(X)
mbk_means_labels_unique = np.unique(mbk.labels_)

In [ ]:
mbk.labels_

In [ ]:
mbk.cluster_centers_

In [ ]:
# Because many blocs contain zero dose of vasopressor or zero dose or IV fluids, put zero 
# values into its own bin and evenly distribute the remaining values.
n_actions = 25
def non_zero_bins(s):
    out, retbins = pd.qcut(s.where(s >= 0.001),4,retbins=True,labels=False)
    retbins = np.insert(retbins,0,float('-inf'))
    return pd.cut(s,retbins,labels=False)

vaso_bins = non_zero_bins(df['max_dose_vaso'])
tev_bins = non_zero_bins(df['cumulated_balance_tev'])
print("Vasopressor bins counts:\n",vaso_bins.value_counts())
print("IV fluid bin counts:\n",tev_bins.value_counts())


In [ ]:
d = {'state': mbk.labels_, 'action':vaso_bins*5 + tev_bins, 'reward': df[results].sum(axis=1), 
     'bloc':df.bloc, 'id':df.icustayid }
mdp = pd.DataFrame(data = d)
episodes = mdp.sort_values(['id','bloc']).groupby('id')
mdp.head(5)

In [ ]:
state_action_count = np.zeros((n_clusters, n_actions))

for i in range(len(mdp)):
    s= mdp.state[i]
    a= mdp.action[i]
    state_action_count[s,a] += 1




In [ ]:
sas_count = np.zeros((n_clusters, n_actions))

i =0
for name, transitions in episodes:
    state = transitions['state'].tolist()
    action = transitions['action'].tolist()
    for i in range(len(state)-2):
        sas_count[state[i],action[i]] += 1

In [ ]:
sas_nul_count = np.count_nonzero(sas_count==0.0)
print ("Number of null items:",sas_nul_count, " out of ", n_clusters*n_actions)
sparsity = 100*(1 - (n_clusters*n_actions - sas_nul_count) / (n_clusters*n_actions)) 
print ("The state-action-sprime matrix is {0:.2f}% sparse".format(sparsity))
distinct_sas_count = sorted(sas_count[np.nonzero(sas_count)],reverse=True)
print ("Total number of distinct state, action, sprime tuples:",len(distinct_sas_count))

In [ ]:
f = np.sum(state_action_count != 0, axis = 1)
f

In [ ]:
nul_count = np.count_nonzero(state_transition_count==0.0)
print ("Number of null transitions:",nul_count, " out of ", n_clusters*n_clusters)
sparsity = 100*(1 - (n_clusters*n_clusters - nul_count) / (n_clusters*n_clusters)) 
print ("The state transitions are {0:.2f}% sparse".format(sparsity))
t_count = sorted(state_transition_count[np.nonzero(state_transition_count)],reverse=True)
print ("Total number of different transitions:",len(t_count))

In [ ]:
plt.hist(f)

In [ ]:
psa = state_action_count/ np.sum(state_action_count, axis = 1)[:, np.newaxis]

In [ ]:
f2 = np.sum(psa>0.1, axis = 1)
plt.hist(f2)

# Random policy

In [ ]:
mdp.groupby('id')

In [ ]:
df.columns